In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

C:\Users\PokeG\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
price = pd.read_csv('Ames_HousePrice.csv')

In [3]:
#function to find outliers for numerical variables based on stdev from mean
def outlier_find(df,cname,numstd):
    for c in cname:
        grl = df[cname]
        grl = np.array(grl)
    
        mean = np.mean(grl, axis=0)
        sd = np.std(grl, axis=0)
        df = df[(df[cname] >= mean+(numstd*sd)) | (df[cname] <= mean-(numstd*sd))]
        
        print(df)

In [4]:
# Removing the outlier on the column 'GrLivArea'

price = price.loc[price["GrLivArea"] != 4676]

###can discuss how much we want to include outliers - additional ones do not help much 

price = price.loc[price["GrLivArea"] != 4676]
price = price.loc[price["SalePrice"] != 755000]
price = price.loc[price["LotArea"] != 159000]
price = price.loc[price["LotArea"] != 215245]
price = price.loc[price["LotArea"] != 215245]
price = price.loc[price["LotArea"] != 115149]
price = price.loc[price["LotArea"] != 164660]
price = price.loc[price["LotArea"] != 70761]

In [5]:
#remove columns with a significant percentage of null values or same values that will give model insignificant value

price_new = price.loc[:, ~price.columns.isin(['Unnamed: 0', 'PID', 'PoolQC', 'PoolArea', 'Fence', 'MiscFeature', 'Alley', 'FireplaceQu', 'MiscVal', 'MoSold', 'YrSold', 
                                              'SaleType', 'SaleCondition'])]
price_new.head()

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,856,126000,30,RL,NaN,7890,Pave,Reg,Lvl,AllPub,...,2.0,399.0,TA,TA,Y,0,0,0,0,166
1,1049,139500,120,RL,42.0,4235,Pave,Reg,Lvl,AllPub,...,1.0,266.0,TA,TA,Y,0,105,0,0,0
2,1001,124900,30,C (all),60.0,6060,Pave,Reg,Lvl,AllPub,...,1.0,216.0,TA,Po,N,154,0,42,86,0
3,1039,114000,70,RL,80.0,8146,Pave,Reg,Lvl,AllPub,...,1.0,281.0,TA,TA,N,0,0,168,0,111
4,1665,227000,60,RL,70.0,8400,Pave,Reg,Lvl,AllPub,...,2.0,528.0,TA,TA,Y,0,45,0,0,0


In [6]:
len(price_new)

2573

In [7]:
pd.set_option("display.max_columns" , None)
null_price_new = price_new.isnull().sum().to_frame()
null_price_new.T

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,0,0,0,0,459,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,14,0,0,0,69,69,71,69,1,70,1,1,1,0,0,0,1,0,0,0,2,2,0,0,0,0,0,0,0,0,127,129,129,1,1,129,129,0,0,0,0,0,0


In [8]:
price_dtypes = price_new.dtypes.to_frame()
price_dtypes.T

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,int64,int64,int64,object,float64,int64,object,object,object,object,object,object,object,object,object,object,object,int64,int64,int64,int64,object,object,object,object,object,float64,object,object,object,object,object,object,object,float64,object,float64,float64,float64,object,object,object,object,int64,int64,int64,float64,float64,int64,int64,int64,int64,object,int64,object,int64,object,float64,object,float64,float64,object,object,object,int64,int64,int64,int64,int64


In [9]:
# NaN imputation

nan_replace = {'BsmtQual': 'No Basement', 'BsmtCond': 'No Basement', 'BsmtExposure': 'No Basement', 'BsmtFinType1': 'No Basement', 'BsmtFinType2': 'No Basement', 
               'GarageFinish': 'No Garage', 'GarageQual': 'No Garage', 'GarageCond': 'No Garage', 'GarageType': 'No Garage'}


price_new.fillna(value = nan_replace, inplace = True)

<ipython-input-9-fbade6c34d2d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_new.fillna(value = nan_replace, inplace = True)


In [10]:
price_new.head()

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,856,126000,30,RL,NaN,7890,Pave,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166
1,1049,139500,120,RL,42.0,4235,Pave,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0
2,1001,124900,30,C (all),60.0,6060,Pave,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0
3,1039,114000,70,RL,80.0,8146,Pave,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111
4,1665,227000,60,RL,70.0,8400,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0


In [11]:
pd.set_option("display.max_columns" , None)
null_price_new = price_new.isnull().sum().to_frame()
null_price_new.T

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,0,0,0,0,459,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,14,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,2,2,0,0,0,0,0,0,0,0,0,129,0,1,1,0,0,0,0,0,0,0,0


In [12]:
len(price_new)

2573

In [13]:
price_new = price_new.astype({'OverallQual': object, 'OverallCond': object, 'MSSubClass': object})

In [14]:
price_dtypes = price_new.dtypes.to_frame()
price_dtypes.T

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,int64,int64,object,object,float64,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,int64,object,object,object,object,object,float64,object,object,object,object,object,object,object,float64,object,float64,float64,float64,object,object,object,object,int64,int64,int64,float64,float64,int64,int64,int64,int64,object,int64,object,int64,object,float64,object,float64,float64,object,object,object,int64,int64,int64,int64,int64


In [15]:
null_vnr = price_new[price_new['Electrical'].isnull()]
null_vnr

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
2442,1394,167500,80,RL,73.0,9735,Pave,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,SLvl,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,NaN,754,640,0,0.0,0.0,2,1,3,1,Gd,7,Typ,0,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0


In [16]:
price_new.dropna(subset = ['MasVnrType', 'Electrical'], inplace = True)

In [17]:
null_price_new = price_new.isnull().sum().to_frame()
null_price_new.T

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,0,0,0,0,454,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,128,0,1,1,0,0,0,0,0,0,0,0


In [18]:
ords = ['OverallQual', 'OverallCond', 'LotShape', 'LandContour', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
        'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive']

for ord in ords:
  print(price_new[ord].value_counts())

5     751
6     666
7     514
8     286
4     195
9      80
3      32
10     18
2      13
1       3
Name: OverallQual, dtype: int64
5    1384
6     495
7     359
8     140
4      88
9      41
3      39
2       8
1       4
Name: OverallCond, dtype: int64
Reg    1630
IR1     847
IR2      69
IR3      12
Name: LotShape, dtype: int64
Lvl    2321
HLS      95
Bnk      94
Low      48
Name: LandContour, dtype: int64
AllPub    2556
NoSewr       2
Name: Utilities, dtype: int64
Gtl    2441
Mod     108
Sev       9
Name: LandSlope, dtype: int64
TA    1630
Gd     827
Ex      71
Fa      30
Name: ExterQual, dtype: int64
TA    2210
Gd     279
Fa      56
Ex      11
Po       2
Name: ExterCond, dtype: int64
TA             1165
Gd             1061
Ex              183
Fa               78
No Basement      69
Po                2
Name: BsmtQual, dtype: int64
TA             2295
Gd              100
Fa               88
No Basement      69
Ex                3
Po                3
Name: BsmtCond, dtype: int64
No    

In [19]:
# ONLY RUN THIS CELL ONCE!

def ord_encoder(col, cats):
  categories = pd.Categorical(price_new[col], categories = cats, ordered = True)

  labels, unique = pd.factorize(categories, sort = True)
  price_new[col] = labels

ord_encoder('OverallQual', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
ord_encoder('OverallCond', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
ord_encoder('LotShape', ['IR3', 'IR2', 'IR1', 'Reg'])
ord_encoder('LandContour', ['Low', 'HLS', 'Bnk', 'Lvl'])
ord_encoder('Utilities', ['ELO', 'NoSeWa', 'NoSeWr', 'AllPub'])
ord_encoder('LandSlope', ['Gtl', 'Mod', 'Sev'])
ord_encoder('ExterQual', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
ord_encoder('ExterCond', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
ord_encoder('BsmtQual', ['No Basement','Po', 'Fa', 'TA', 'Gd', 'Ex'])
ord_encoder('BsmtCond', ['No Basement','Po', 'Fa', 'TA', 'Gd', 'Ex'])
ord_encoder('BsmtExposure', ['No Basement','No','Mn', 'Av','Gd'])
ord_encoder('BsmtFinType1', ['No Basement', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'])
ord_encoder('BsmtFinType2', ['No Basement', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'])
ord_encoder('HeatingQC', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
ord_encoder('KitchenQual', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
ord_encoder('Functional', ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'])
ord_encoder('GarageFinish', ['No Garage', 'Unf', 'RFn', 'Fin'])
ord_encoder('GarageQual', ['No Garage', 'Po', 'Fa', 'TA', 'Gd', 'Ex'])
ord_encoder('GarageCond', ['No Garage', 'Po', 'Fa', 'TA', 'Gd', 'Ex'])
ord_encoder('PavedDrive', ['N', 'P', 'Y'])

In [20]:
ords = ['OverallQual', 'OverallCond', 'LotShape', 'LandContour', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
        'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive']

for ord in ords:
  print(price_new[ord].value_counts())

4    751
5    666
6    514
7    286
3    195
8     80
2     32
9     18
1     13
0      3
Name: OverallQual, dtype: int64
4    1384
5     495
6     359
7     140
3      88
8      41
2      39
1       8
0       4
Name: OverallCond, dtype: int64
3    1630
2     847
1      69
0      12
Name: LotShape, dtype: int64
3    2321
1      95
2      94
0      48
Name: LandContour, dtype: int64
 0    2556
-1       2
Name: Utilities, dtype: int64
0    2441
1     108
2       9
Name: LandSlope, dtype: int64
1    1630
2     827
3      71
0      30
Name: ExterQual, dtype: int64
2    2210
3     279
1      56
4      11
0       2
Name: ExterCond, dtype: int64
3    1165
4    1061
5     183
2      78
0      69
1       2
Name: BsmtQual, dtype: int64
3    2295
4     100
2      88
0      69
5       3
1       3
Name: BsmtCond, dtype: int64
1    1694
3     343
4     235
2     215
0      71
Name: BsmtExposure, dtype: int64
6    742
1    699
5    394
3    264
4    251
2    139
0     69
Name: BsmtFinType1, dtype: in

In [21]:
# ONLY RUN THIS CELL ONCE!
from sklearn import preprocessing

def nom_encoder(col):
  global price_new

  le = preprocessing.LabelEncoder()
  le.fit(price_new[col])
  price_new = price_new.apply(le.fit_transform)
  return price_new

noms = ['MSSubClass', 'MSZoning', 'Street', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 
        'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType']

for nom in noms:
  nom_encoder(nom)

In [22]:
null_price_new = price_new.isnull().sum().to_frame()
null_price_new.T

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
len(null_price_new)

69

In [24]:
price.head()

,Unnamed: 0,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
1,2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
2,3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
3,4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
4,5,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


In [25]:
price_new.head()

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,70,197,1,5,126,457,1,3,3,1,0,0,21,2,2,0,2,5,5,44,0,1,0,13,14,2,0,1,2,1,3,3,1,3,120,1,0,518,251,1,2,1,3,189,0,0,1,0,1,0,2,1,2,2,6,1,5,29,1,2,145,3,3,2,0,0,0,0,51
1,197,260,11,5,18,134,1,3,3,1,4,0,7,2,2,4,2,4,4,87,34,1,0,6,6,1,107,2,2,1,4,3,2,6,365,5,131,64,403,1,2,1,3,362,0,0,1,0,2,0,2,1,3,3,6,0,1,73,3,1,48,3,3,2,0,93,0,0,0
2,160,191,1,1,36,236,1,3,3,1,4,0,11,2,2,0,2,4,8,36,57,3,0,8,8,2,0,2,2,0,3,3,1,5,512,1,0,61,234,1,4,1,3,319,0,0,0,0,1,0,2,1,3,3,6,0,5,20,1,1,21,3,1,0,109,0,16,1,0
3,188,149,6,5,56,509,1,3,3,1,0,0,20,2,2,0,5,3,7,11,53,1,0,8,8,2,0,2,3,0,2,3,1,1,0,1,0,336,26,1,3,1,3,87,23,0,0,0,1,0,2,1,2,4,6,0,5,30,1,1,57,3,3,0,0,0,93,0,16
4,701,585,5,5,46,561,1,3,3,1,4,0,17,2,2,0,5,7,5,104,51,1,0,12,13,2,0,2,2,2,4,3,1,6,435,1,0,119,211,1,4,1,3,152,338,0,1,0,2,1,3,1,3,4,6,0,1,90,3,2,259,3,3,2,0,35,0,0,0


In [26]:
#price_new.to_csv('Ames_Price_Preprocessed_Tree.csv')